In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import datetime as dt
import numpy as np
import pyart

from cappi.nexrad_wave_form import compute_nexrad_wave_form
from cappi.prune_nexrad import prune_nexrad
from cappi.helpers import pyart_to_xarray_vol
from cappi.xy_cappi import raw_3dvol_to_xy, azran_to_xy
from config import _EXAMPLEDATA_DIR

# OPTION 1: COMPUTE THE AZRAN CAPPI @ -10°C

In [2]:
# make and save AzRan CAPPI the Python way

filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KOAX20140603_213649_V06.gz')
cappi_target_height_m = 5188.0   # -10°C isotherm height KOAX 20140603
x_trg = np.arange(-200*1000, 200*1000+1, 1000)
y_trg = np.arange(-200*1000, 200*1000+1, 1000)
field = 'reflectivity'

radar_vol = pyart.io.read_nexrad_archive(filename)
nexrad_wave_form = compute_nexrad_wave_form(radar_vol)

prune_actions = ['surv', 'volume']
prune_vol = prune_nexrad(prune_actions, radar_vol)

ds_vol = pyart_to_xarray_vol(prune_vol, ['reflectivity'], reindex_angle=False)

# note that rawvol_to_cartesian takes the target height in m AMSL - needs to 
# be taken into account (not done yet below)
ds_grid = raw_3dvol_to_xy(ds_vol, x_trg, y_trg, cappi_target_height_m)

#out_filepath = Path(_EXAMPLEDATA_DIR, 'test', 'CAPPIneg10C_00.00_KOAX_20140603_213649.nc')
# save to netcdf file
#pyart.io.write_cfradial(out_filepath, cappi)

In [3]:
# load AzRan CAPPIs from the docker output
# radar_dir = Path(_EXAMPLEDATA_DIR, 'test', 'KOAX20140603')
# timestep = dt.datetime(2014, 6, 3, 21, 36, 50)
# 
# cappi_docker = read_azran(
#     radar_dir, timestep, subdir="CAPPI/raw", cappi_tlevels=["neg10"])

# read 3D volumes instead
# dir_day = Path(NEXRAD_DIR, "KFWS20160411")
# filename_refl = Path(dir_day, "3D", "Reflectivity3D_20160411-210750.netcdf")
# radar = read_3d_netcdf(filename_refl)

In [5]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121)
# display_docker = pyart.graph.RadarDisplay(cappi_docker)
# display_docker.plot('reflectivity', sweep=0, ax=ax1, cmap='NWSRef', vmin=-20,
#                     vmax=75, mask_outside=True, title_flag=False, 
#                     colorbar_flag=False)
# display_docker.set_limits([0, 100], [-35, 65])
# ax1.set_title('-10°C CAPPI from docker')
# display_docker.plot_colorbar(shrink=0.75, label='Z (dBZ)')

ax2 = fig.add_subplot(122)
ds_grid['reflectivity'].plot(x="x", y="y", cmap='NWSRef', vmin=-20, vmax=75,
                             xlim=(0*1000, 100*1000), ylim=(-35*1000, 65*1000))
ax2.set_title('-10°C CAPPI from Python')

for ax in [ax1, ax2]:
    ax.set_aspect('equal')
    
plt.tight_layout()